In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

# Basic Convnet

In [28]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
    # input data
    tf_train_dataset = tf.placeholder(tf.float32, shape= (batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dateset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal([image_size//4 * image_size//4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # model
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
                            
    
    # training
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # Prediction and accuracy for training, valid, test
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dateset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [29]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.093352
Minibatch accuracy: 18.8%
Validation accuracy: 8.5%
Minibatch loss at step 50: 1.645082
Minibatch accuracy: 43.8%
Validation accuracy: 49.8%
Minibatch loss at step 100: 1.073726
Minibatch accuracy: 62.5%
Validation accuracy: 64.2%
Minibatch loss at step 150: 0.304366
Minibatch accuracy: 93.8%
Validation accuracy: 75.3%
Minibatch loss at step 200: 0.833680
Minibatch accuracy: 68.8%
Validation accuracy: 76.5%
Minibatch loss at step 250: 0.917385
Minibatch accuracy: 75.0%
Validation accuracy: 78.3%
Minibatch loss at step 300: 0.344657
Minibatch accuracy: 87.5%
Validation accuracy: 79.0%
Minibatch loss at step 350: 0.550614
Minibatch accuracy: 93.8%
Validation accuracy: 76.6%
Minibatch loss at step 400: 0.362523
Minibatch accuracy: 87.5%
Validation accuracy: 80.3%
Minibatch loss at step 450: 0.800220
Minibatch accuracy: 81.2%
Validation accuracy: 79.2%
Minibatch loss at step 500: 0.777600
Minibatch accuracy: 75.0%
Validation accuracy: 80.2%
Mi

# Add Pooling

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
start_learning_rate = 0.05
dropout = 0.8
num_steps = 2001

graph = tf.Graph()

with graph.as_default():
    
    # input data
    tf_train_dataset = tf.placeholder(tf.float32, shape= (batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dateset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal([image_size//4 * image_size//4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # model
    def model(data, train = True):
        
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1,1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        pool = tf.nn.max_pool(hidden, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
        
        conv = tf.nn.conv2d(pool, layer2_weights, [1, 1,1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        pool = tf.nn.max_pool(hidden, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
        
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        if (train):
            hidden = tf.nn.dropout(hidden, dropout)
            
        return tf.matmul(hidden, layer4_weights) + layer4_biases
                            
    
    # training
    logits = model(tf_train_dataset)
    # print(logits[0:10])
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # optimizer
    learning_rate = tf.train.exponential_decay(start_learning_rate, num_steps, 2001, 0.95, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
    
    # Prediction and accuracy for training, valid, test
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dateset,False))
    test_prediction = tf.nn.softmax(model(tf_test_dataset,False))

In [8]:


with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.308415
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.912343
Minibatch accuracy: 12.5%
Validation accuracy: 34.2%
Minibatch loss at step 100: 1.508676
Minibatch accuracy: 43.8%
Validation accuracy: 57.1%
Minibatch loss at step 150: 0.630421
Minibatch accuracy: 81.2%
Validation accuracy: 74.2%
Minibatch loss at step 200: 1.041650
Minibatch accuracy: 62.5%
Validation accuracy: 77.8%
Minibatch loss at step 250: 1.555207
Minibatch accuracy: 50.0%
Validation accuracy: 75.3%
Minibatch loss at step 300: 0.537474
Minibatch accuracy: 87.5%
Validation accuracy: 79.0%
Minibatch loss at step 350: 0.426450
Minibatch accuracy: 87.5%
Validation accuracy: 77.7%
Minibatch loss at step 400: 0.345033
Minibatch accuracy: 87.5%
Validation accuracy: 80.6%
Minibatch loss at step 450: 0.741173
Minibatch accuracy: 81.2%
Validation accuracy: 80.3%
Minibatch loss at step 500: 0.694278
Minibatch accuracy: 87.5%
Validation accuracy: 81.2%
M

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
start_learning_rate = 0.05
dropout = 0.8
num_steps = 1001
ACC = 90.2%

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
start_learning_rate = 0.05
dropout = 0.8
num_steps = 1001
ACC = 91.9%